# Acquire and Analyze Project

# Initialize Twitter API 

In [1]:
import tweepy
import pandas as pd
import tweepy
import os
import datetime
import re
from pprint import pprint
from nltk.corpus import stopwords
sw = stopwords.words('english')
from string import punctuation
from nltk.probability import FreqDist

In [2]:
from fanok_api_keys import api_key, api_key_secret, bearer_token,access_token, access_token_secret

In [3]:
#authentication

auth = tweepy.OAuthHandler(api_key,api_key_secret)
auth.set_access_token(access_token, access_token_secret)

#API Connection

api = tweepy.API(auth)

# Pulling our text data (Republican)

In [4]:
user = 'tedcruz'
repub_data = []
tweets = tweepy.Cursor(api.user_timeline,screen_name = user, 
                        count = 1000, 
                        tweet_mode = 'extended').items()

for tweet in tweets:
    repub_data.append([tweet.user.screen_name, tweet.full_text])

In [5]:
user = 'mtgreenee'

tweets = tweepy.Cursor(api.user_timeline,screen_name = user, 
                        count = 1000, 
                        tweet_mode = 'extended').items()

for tweet in tweets:
    repub_data.append([tweet.user.screen_name, tweet.full_text])


In [6]:
user = 'laurenboebert'

tweets = tweepy.Cursor(api.user_timeline,screen_name = user, 
                        count = 1000, 
                        tweet_mode = 'extended').items()

for tweet in tweets:
    repub_data.append([tweet.user.screen_name, tweet.full_text])

In [7]:
keywords = '#MAGA'

tweets = tweepy.Cursor(api.search_tweets,q = keywords, 
                        count = 1000, 
                        tweet_mode = 'extended').items(1000)

for tweet in tweets:
    repub_data.append([keywords,tweet.full_text])

TooManyRequests: 429 Too Many Requests
88 - Rate limit exceeded

# Pulling our text data (Democratic)

In [7]:
user = 'AOC'
dem_data = []
tweets = tweepy.Cursor(api.user_timeline,screen_name = user, 
                        count = 1000, 
                        tweet_mode = 'extended').items()

for tweet in tweets:
    dem_data.append([tweet.user.screen_name, tweet.full_text])

In [8]:
user = 'potus'

tweets = tweepy.Cursor(api.user_timeline,screen_name = user, 
                        count = 1000, 
                        tweet_mode = 'extended').items()

for tweet in tweets:
    dem_data.append([tweet.user.screen_name, tweet.full_text])

In [9]:
user = 'HillaryClinton'

tweets = tweepy.Cursor(api.user_timeline,screen_name = user, 
                        count = 1000, 
                        tweet_mode = 'extended').items()

for tweet in tweets:
    dem_data.append([tweet.user.screen_name, tweet.full_text])

In [14]:
keywords = '#Democrat'

tweets = tweepy.Cursor(api.search_tweets,q = keywords, 
                        count = 1000, 
                        tweet_mode = 'extended').items()

for tweet in tweets:
    dem_data.append([keywords,tweet.full_text])

TooManyRequests: 429 Too Many Requests
88 - Rate limit exceeded

# Building our Republican DF

In [10]:
columns = ['User/Keyword', 'Tweets']
republican_df = pd.DataFrame(repub_data, columns = columns)

In [11]:
republican_df

,User/Keyword,Tweets
0,tedcruz,surprisingly accurate https://t.co/J6mUoUI1tI
1,tedcruz,Leftists want America to produce less oil and ...
2,tedcruz,"RT @RichSementa: Sen @tedcruz: ""The fish rots ..."
3,tedcruz,RT @GuyBensonShow: .@SenTedCruz (R-TX): If We ...
4,tedcruz,RT @EpochTimes: “Voting is a distinct right an...
...,...,...
9642,laurenboebert,Remember that time when a renegade political p...
9643,laurenboebert,"Anthony Fauci, whom I refuse to refer to as “d..."
9644,laurenboebert,Fauci's new book was just scrubbed on Amazon a...
9645,laurenboebert,The students at the makeshift schools on the b...


# Building our Democratic DF

In [12]:
columns = ['User/Keyword', 'Tweets']
democratic_df = pd.DataFrame(dem_data, columns = columns)

In [13]:
democratic_df

,User/Keyword,Tweets
0,AOC,After Trump elevated anti-immigrant &amp; anti...
1,AOC,.@laurenboebert you have played a major role i...
2,AOC,RT @SIfill_: You are witnessing one of those r...
3,AOC,RT @nycsouthpaw: A letter to Chief Justice Rob...
4,AOC,RT @MondaireJones: How anyone could read this ...
...,...,...
8284,HillaryClinton,RT @NYGovCuomo: Today we are sending a strong ...
8285,HillaryClinton,I’ll be joining @NYGovCuomo as he lays out his...
8286,HillaryClinton,"To the women sworn in to Congress today, the m..."
8287,HillaryClinton,I'm grateful to everyone who brought light int...


# Cleaning Tweets for Analysis

In [14]:
punct_set = punctuation

In [15]:
repub_names = ['tedcruz', 'mtgreene', 'laurenboebert']

In [16]:
#First let's make both our lists long strings

repub_data_list= [','.join(w) for w in repub_data]

#This gives us a long string of text
repub_data_string = ''.join(repub_data_list)

#this tokenizes our tweet data
repub_data_tokenized_string = [w.lower() for w in repub_data_string.split() if w.isalpha() and w not in repub_names]


In [72]:
#repub_data_string
#repub_data_tokenized_string

In [73]:
dem_names = ['AOC', 'POTUS', 'HillaryClinton']

In [80]:
#First let's make both our lists long strings

dem_data_list= [','.join(w) for w in dem_data]

#This gives us a long string of text
dem_data_string = ''.join(dem_data_list)

#this tokenizes our tweet data
dem_data_tokenized_string = [w.lower() for w in dem_data_string.split() if w.isalpha() and w not in dem_names]

In [81]:
#dem_data_string
#dem_data_tokenized_string

# Analysis 1: Most Popular Words

In [18]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [39]:
repub_freqdist = FreqDist()

for word in repub_data_tokenized_string:
    repub_freqdist[word]+=1  

In [40]:
repub_top_words = repub_freqdist.most_common(20) 

In [66]:
columns = ['word', 'count']

repub_words_df = pd.DataFrame(repub_top_words,columns = columns) 

In [67]:
repub_words_df.values()

TypeError: 'numpy.ndarray' object is not callable

In [63]:
df = repub_words_df
plt.figure(figsize=(10,5))
sns.barplot(df.values, df.index, alpha=0.8)
plt.title('Most Popular Words: Republican')
plt.ylabel('Words from Tweets', fontsize=12)
plt.xlabel('Word Count', fontsize=12)
plt.show()

AttributeError: 'bool' object has no attribute 'all'

<Figure size 1000x500 with 0 Axes>